In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import matplotlib.image as image

import sys
sys.path.append('./')
from utils import Dummyfier, prepare, evaluate_clf, uplift

from IPython.core.debugger import set_trace

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [92]:
test =  pd.read_csv('./test_data_04_2018/test_04_2018.csv/CD_ALL_2018-03-30.csv', sep='>', na_values='<null>', 
                     parse_dates=['LAST_CARD_STATUS_DT', 'LAST_CONTR_STATUS_DT'], encoding='latin-1')
print('Dataset size: ', test.shape)
#test = test.set_index(['CONTRACT_REF', 'CLIENT_ID'])
test.head(2)

Dataset size:  (102532, 369)


,CLIENT_ID,CONTRACT_REF,BANK,label,TOTAL_BAL_NEG_CHNG_MAX_180,TOTAL_BAL_NEG_CHNG_MAX_31,TOTAL_BAL_NEG_CHNG_MAX_7,TOTAL_BAL_NEG_CHNG_MAX_90,TOTAL_BAL_NEG_CHNG_MIN_180,TOTAL_BAL_NEG_CHNG_MIN_31,...,CL_AMT_ZP,CL_CCY_DOP,CL_AMT_DOP,FL_DIR_UCH,FL_4P,FL_DOC_PROFIT,REP_POPULATION,FL_PENS_DOC,FL_FRNG_PASSP,BIRTH_DT
0,103016707,273-P-90421929,BBKK,1,0.0,0.0,0.0,0.0,0.0,0.0,...,20000.0,NaN,12000.0,N,N,N,746400.0,N,N,1952-08-16 00:00:00
1,104531162,271-P-21418527,BBKK,1,0.0,0.0,0.0,0.0,0.0,0.0,...,11000.0,RUR,0.0,N,N,N,1255100.0,N,N,1956-08-17 00:00:00


In [93]:
#test['BIRTH_DT']

In [94]:
train = pd.read_csv('./train_data/train_dataset.csv', sep='>', na_values='<null>', 
                 parse_dates=['START_DATE', 'LAST_CARD_STATUS_DT', 'LAST_CONTR_STATUS_DT', 'BIRTH_DATE'])
print('Dataset size: ', train.shape)
train.head(2)

Dataset size:  (114720, 215)


,CONTRACT_REF,CLIENT_ID,CONTROL_GROUP_FLG,START_DATE,TARGET,11_AddrCity2,11_AddrCountry,12_AddrCity2,12_AddrCountry,13_AddrCity2,...,TXN_TP_POS_N_180,TXN_TP_POS_N_31,TXN_TP_POS_N_90,TXN_TP_POS_SUM_180,TXN_TP_POS_SUM_31,TXN_TP_POS_SUM_90,TXN_TP_Purposeful credit_MAX_180,TXN_TP_Purposeful credit_MEAN_180,TXN_TP_Purposeful credit_N_180,TXN_TP_Purposeful credit_SUM_180
0,273-P-99431598,105098373,N,2017-01-23,1,ГОРЯЧИЙКЛЮЧ,RU,ГОРЯЧИЙКЛЮЧ,RU,КАМЫШИН,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,274-P-23967129,101763714,N,2017-04-08,1,МУРМАНСК,RU,МУРМАНСК,RU,АХТУБИНСК,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
# cols which are in both datasets
cols_intersect = list(set(train.columns).intersection(set(test.columns)))
print(len(cols_intersect))
print(len(set(train.reset_index().columns)))
print(len(set(test.reset_index().columns)))

159
216
370


In [96]:
train['TARGET'].value_counts(normalize=True)

0    0.955483
1    0.044517
Name: TARGET, dtype: float64

In [97]:
#train = train[['TARGET'] + cols_intersect]
#test = test[cols_intersect]
print(train.shape)
print(test.shape)

(114720, 215)
(102532, 369)


In [105]:
def fl_pens(series):
    if series['FL_PENS'] == 0:
        return 0
    elif series['FL_PENS_DOC'] == 0:
        return 1
    else:
        return 2

def arrange_on_circle(value, unique):
    """arrange some categorial feature on circle for linear models"""
    angle = np.linspace(0, 2*np.pi, unique)[value]
    return np.sin(angle), np.cos(angle)

def prepare(df):

    df['AGE'] = (datetime.today() - df['BIRTH_DATE']).apply(lambda x: int(x.days/365.25))
    df['SIN_BD_MONTH'], df['COS_BD_MONTH'] =zip(*df['BIRTH_DATE'].apply(lambda x: arrange_on_circle(x.month-1, 12)))
    #df['ZODIAC'] = df['BIRTH_DATE'].apply(zodiac)

    df['FL_PEN_DOC'] = df.apply(lambda row: fl_pens(row), axis=1)

    df['LAST_CARD_STATUS_TIME'] = (df['START_DATE'] - df['LAST_CARD_STATUS_DT']).dt.days
    df['LAST_CONTR_STATUS_TIME'] = (df['START_DATE'] - df['LAST_CONTR_STATUS_DT']).dt.days
    df['CODE_67_90'] = df['CODE_67_N_90'].apply(lambda x: 0 if x == 0 else 1)

    # change old features:
    df['CODE_67_N_180'] = df['CODE_67_N_180'].apply(lambda x: 3.5 if x >= 3 else x)
    df['IS_LOYALTY_AUTO'] = df['IS_LOYALTY_AUTO'].apply(lambda x: 0 if x == 0 else 1)
    df['QTY_CHLD'] = df['QTY_CHLD'].apply(lambda x: 4 if x >= 4 else x)
#
    ##### 
    education = {
        'POS': 7, 
        'SEC': 11, 
        'TEC': 11 + 1, 
        'UND': 11 + 3, 
        'HIG': 11 + 5, 
        'TWO': 11 + 5 + 3, 
                }
    df['CL_EDU'] = df['CL_EDU'].map(education)

   
    card_status = {
        14: 14,
        176: 176,
        100000022: 176,
        182: 182,
        74: 74,
        63: 63,
        98: 98,
        100000100: 98,
        100000094: 98,
        100000087: 98,
        100000496: 176,
        100000576: 176
    }

    df['LAST_CARD_STATUS'] = df['LAST_CARD_STATUS'].map(card_status)

   
    sex = {
        'F': 0, 
        'M': 1,
        }
    df['REP_SEX'] = df['REP_SEX'].map(sex)

    #### 
    binary = {
        'N': 0,
        'Y': 1,
    }
    df['FL_GOLD'] = df['FL_GOLD'].map(binary)
    df['FL_ZP'] = df['FL_ZP'].map(binary)
    df['FL_PENS'] = df['FL_PENS'].map(binary)
    df['FL_SOC'] = df['FL_SOC'].map(binary)
    df['FL_P24'] = df['FL_P24'].map(binary)
    df['FL_MOBB'] = df['FL_MOBB'].map(binary)
    df['FL_SMSI'] = df['FL_SMSI'].map(binary)
    df['CL_AUTO_EX'] = df['CL_AUTO_EX'].map(binary)
    df['FL_4P'] = df['FL_4P'].map(binary)
    df['FL_DOC_PROFIT'] = df['FL_DOC_PROFIT'].map(binary)
    df['FL_PENS_DOC'] = df['FL_PENS_DOC'].map(binary)
    df['FL_FRNG_PASSP'] = df['FL_FRNG_PASSP'].map(binary)
    df['FL_FRGN_TRIP'] = df['FL_FRGN_TRIP'].map(binary)

    try:
        df['CONTROL_GROUP_FLG'] = df['CONTROL_GROUP_FLG'].map(binary)
    except:
        pass
    
    df.loc[:, 'CARD_STATUS_14'] = df.loc[:, 'CARD_STATUS_14'].fillna(0)
    df.loc[:, 'CARD_STATUS_176'] = df.loc[:, 'CARD_STATUS_176'].fillna(0)
    df.loc[:, 'CL_AMT_DOP'] = df.loc[:, 'CL_AMT_DOP'].fillna(0)

    return df    

In [106]:
dummy = [u'CL_FAM_ST', u'CL_OFF_POS', u'CL_OWN_TYPE', u'LAST_CARD_STATUS', 'CL_ESTATE_TYPE', 
         'CL_INDUSTR', 'CL_SOC_ST', 'ZODIAC']
d = Dummyfier(dummy, drop_old=True)

Xy_train = prepare(train)
Xy_train = d.fit_transform(Xy_train)

na_cols_train = Xy_train.isna().any()
na_cols_train = na_cols_train[na_cols_train==True].index.tolist()
Xy_train = Xy_train.fillna(value=Xy_train[na_cols_train].mean())

y_train = Xy_train['TARGET'].values.flatten()

identity = ['CONTRACT_REF', 'CLIENT_ID', 'SCENARIO_NAME_UNI', 'START_DATE']
X_train = Xy_train.drop(['TARGET'] + identity, axis=1)


X_test = prepare(test)
X_test = d.transform(X_test)

na_cols_test = X_test.isna().any()
na_cols_test = na_cols_test[na_cols_test==True].index.tolist()
X_test = X_test.fillna(value=X_test[na_cols_test].mean())

X_test = X_test.drop('START_DATE', axis=1)
X_test['CONTROL_GROUP_FLG'] = 0

X_test = X_test[X_train.columns.tolist()]

ValueError: The listed features are not contained in the data

In [88]:
X_test = prepare(test)
#X_test = d.transform(X_test)


ValueError: labels ['CODE_17_MEAN_180' 'CODE_26_MEAN_180' 'CODE_26_SUM_180' 'CODE_2_MEAN_180'
 'CODE_35_MEAN_180' 'CODE_35_SUM_180' 'CODE_46_MEAN_180'
 'TXN_CHN_Our ATM_MEAN_180' 'TXN_CHN_Our ATM_N_180' 'TXN_TP_ATM_MEAN_180'
 'TXN_TP_ATM_N_180' 'TXN_TP_MT_MAX_180' 'TXN_TP_MT_MEAN_180'
 'TXN_TP_MT_N_180' 'TXN_TP_POS_MAX_90' 'TXN_TP_POS_MEAN_31'
 'TXN_TP_POS_MEAN_90' 'TXN_TP_POS_N_31' 'TXN_TP_POS_N_90'
 'TXN_TP_POS_SUM_90'] not contained in axis